In [1]:
import requests
from bs4 import BeautifulSoup


url_stage_1 = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'
response_stage_1 = requests.get(url_stage_1)
html_stage_1 = response_stage_1.text


soup_stage_1 = BeautifulSoup(html_stage_1, 'html.parser')
players_links = []

allRows = soup_stage_1.select('table.engineTable tbody tr.data1')
for row in allRows:
    tds = row.find_all('td')
    rowURL = "https://www.espncricinfo.com" + tds[6].find('a')['href']
    players_links.append(rowURL)


bowling_summaries = []

for link in players_links:
    response_stage_2 = requests.get(link)
    html_stage_2 = response_stage_2.text
    soup_stage_2 = BeautifulSoup(html_stage_2, 'html.parser')

    # Extract match information
    match = soup_stage_2.find('div', text="Match Details").find_next_siblings()
    team1 = match[0].find('span').text.replace(" Innings", "")
    team2 = match[1].find('span').text.replace(" Innings", "")
    matchInfo = f"{team1} Vs {team2}"

    # Extract bowling summary
    tables = soup_stage_2.select('div table.ds-table')
    firstInningRows = tables[1].select('tbody tr')
    secondInningRows = tables[3].select('tbody tr')

    def extract_bowling_data(rows, bowling_team):
        for row in rows:
            tds = row.select('td')
            bowling_summaries.append({
                "match": matchInfo,
                "bowlingTeam": bowling_team,
                "bowlerName": tds[0].select('a span')[0].text.replace('\xa0', ''),
                "overs": tds[1].text,
                "maiden": tds[2].text,
                "runs": tds[3].text,
                "wickets": tds[4].text,
                "economy": tds[5].text,
                "0s": tds[6].text,
                "4s": tds[7].text,
                "6s": tds[8].text,
                "wides": tds[9].text,
                "noBalls": tds[10].text
            })

    extract_bowling_data(firstInningRows, team2)
    extract_bowling_data(secondInningRows, team1)

# Printing the first few records as an example
for record in bowling_summaries[:5]:
    print(record)
